In [ ]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import numpy as np
m=3
mesh = StructuredTesseraktMesh(m,m,m,m)

In [ ]:
from methodsnm.fes import *
fes = P1_Tesserakt_Space(mesh)

## Verification Example for the timeintegral: Solving the ODE *uₜ = t²*

In the following, we consider the simple ordinary differential equation:

$
\frac{\partial u}{\partial t} = t^2
$

This problem is purely time-dependent and serves as a minimal test case to verify the correct implementation of the time integral assembling procedure in the finite element framework.


In [ ]:
from methodsnm.forms import *
from methodsnm.formint import *
from numpy import pi, cos ,sin

blf = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf += TimeIntegral(c)
blf.assemble()

lf = LinearForm(fes)
f = GlobalFunction(lambda x: pi*cos(pi*x[3]), mesh = mesh)
lf += SourceIntegral(f)
lf.assemble()

In [ ]:
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

uh = FEFunction(fes)
f = lambda x: sin(pi*x[3])
uh._set(f, True)
res = lf.vector - blf.matrix.dot(uh.vector)

from scipy.sparse.linalg import spsolve
from methodsnm.solver import solve_on_freedofs
freedofs = list_diff(mesh.vertices,mesh.bndry_vertices)
#uh.vector = spsolve(blf.matrix, lf.vector)
uh.vector += solve_on_freedofs(blf.matrix,res,freedofs)
from methodsnm.forms import compute_difference_L2

uex =  GlobalFunction(f, mesh = mesh)
l2diff = compute_difference_L2(uh, uex, mesh, intorder = 5)

print("l2diff =", l2diff)


## Verification Example for the laplacian withouttime: Solving the ODE $u_{xx} = x²$

In the following, we consider the simple ordinary differential equation:

$
-\Delta u(x) = f(x), \quad x \in \Omega
$

This problem is purely location-dependent and serves as a minimal test case to verify the correct implementation of the time integral assembling procedure in the finite element framework.

In [ ]:
blf3 = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf3 += LaplaceIntegral_without_time(c)
blf3.assemble()

lf3 = LinearForm(fes)
f3 = GlobalFunction(lambda x: 3*pi*cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2]), mesh = mesh)
lf3 += SourceIntegral(f3)
lf3.assemble()

def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

uh3 = FEFunction(fes)
f4 = lambda x: cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])
uh3._set(f4, True)
res = lf3.vector - blf3.matrix.dot(uh3.vector)

from methodsnm.solver import solve_on_freedofs
freedofs = list_diff(mesh.vertices,mesh.bndry_vertices)
uh3.vector += solve_on_freedofs(blf3.matrix,res,freedofs)
from methodsnm.forms import compute_difference_L2

uex =  GlobalFunction(f4, mesh = mesh)
l2diff = compute_difference_L2(uh3, uex, mesh, intorder = 5)
print(l2diff)

## Verification Example: Solving the parabolic PDE 

In the following, we consider the simple ordinary differential equation:

$-\Delta u + \frac{\partial u}{\partial t} = f$

This problem is the mix off the two problems before and serves as a minimal test case to verify the correct implementation of the time integral + laplaceintegral assembling procedure in the finite element framework.

In [ ]:
from methodsnm.forms import *
from methodsnm.formint import *
from numpy import pi, cos ,sin

blf3 = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf3 += TimeIntegral(c)
blf3 += LaplaceIntegral_without_time(c)
blf3.assemble()

lf3 = LinearForm(fes)
f3 = GlobalFunction(lambda x: 0, mesh = mesh)
lf3 += SourceIntegral(f3)
lf3.assemble()

def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

top = mesh.filter_bndry_points("max",3)
bndry = list_diff(mesh.bndry_vertices,top)

uh = FEFunction(fes)
f = lambda x: sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2])
uh._set(f, bndry)
res = lf3.vector - blf3.matrix.dot(uh.vector)

from scipy.sparse.linalg import spsolve
from methodsnm.solver import solve_on_freedofs
freedofs = list_diff(mesh.vertices,bndry)
#uh.vector = spsolve(blf.matrix, lf.vector)
uh.vector += solve_on_freedofs(blf3.matrix,res,freedofs)

from methodsnm.forms import compute_difference_L2

uex =  GlobalFunction(lambda x: sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2])*np.exp(-3*pi**2*x[3]), mesh = mesh)
l2diff = compute_difference_L2(uh, uex, mesh, intorder = 5)

print("l2diff =", l2diff)